# _Get_ TMDB _movies_ _details_

In [1]:
import os
from urllib.parse import urljoin

import dotenv
import pandas as pd
import requests

from utils import tmdb

dotenv.load_dotenv()

TMDB_BASE_URL: str = "https://api.themoviedb.org/3/"
TMDB_REQUEST_HEADER: dict[str, str] = {"Authorization": f"Bearer {os.getenv('TMDB_BEARER_TOKEN')}"}

In [2]:
IDS_ANCINE_TMDB: pd.Series = pd.read_csv(
    "../data/movies-ids-ancine-tmdb.csv",
    index_col="id_ancine",
    dtype_backend="pyarrow",
).squeeze("columns")

In [3]:
details_tmdb: pd.DataFrame = pd.DataFrame(None, index=IDS_ANCINE_TMDB)

In [4]:
URL: str = urljoin(TMDB_BASE_URL, "movie/{}")

with requests.Session() as session:
    session.headers.update(TMDB_REQUEST_HEADER)
    for index, row in details_tmdb.iterrows():
        response: requests.Response = session.get(URL.format(index))
        response.raise_for_status()
        content: dict = response.json()
        details_tmdb.at[index, "countries"] = tmdb.extract_movie_countries(content)
        details_tmdb.at[index, "genres"] = tmdb.extract_movie_genres(content)
        details_tmdb.at[index, "runtime"] = tmdb.extract_movie_runtime(content)

In [ ]:
URL: str = urljoin(TMDB_BASE_URL, "movie/{}/credits")

with requests.Session() as session:
    session.headers.update(TMDB_REQUEST_HEADER)
    for index, row in details_tmdb.iterrows():
        response: requests.Response = session.get(URL.format(index))
        response.raise_for_status()
        content: dict = response.json()
        details_tmdb.at[index, "credits_number"] = tmdb.extract_movie_credits_number(content)

In [ ]:
details_tmdb: pd.DataFrame = (
    details_tmdb.reindex(columns=details_tmdb.columns.sort_values())
    .convert_dtypes(dtype_backend="pyarrow")
    .dropna(how="all")
)

details_tmdb.to_csv("../data/movies-details-tmdb.csv")